---
title: "Kaggle_exercise_04"
author: "minstar"
date: "2023-05-18"
categories: [python]
---
kaggle 문제풀이 04
https://www.kaggle.com/code/lsiina/exercise-manipulating-geospatial-data/edit

# 소개

당신은 스타벅스의 빅데이터 분석가입니다 ([실제로 존재하는 직업입니다!](https://www.forbes.com/sites/bernardmarr/2018/05/28/starbucks-using-big-data-analytics-and-artificial-intelligence-to-boost-performance/#130c7d765cdc)). 당신은 [스타벅스 리저브 로스터리](https://www.businessinsider.com/starbucks-reserve-roastery-compared-regular-starbucks-2018-12#also-on-the-first-floor-was-the-main-coffee-bar-five-hourglass-like-units-hold-the-freshly-roasted-coffee-beans-that-are-used-in-each-order-the-selection-rotates-seasonally-5)에 다음 매장을 찾기 위해 캘리포니아 주의 다양한 군지역의 인구 통계를 조사할 것입니다.

<center>
<img src="https://storage.googleapis.com/kaggle-media/learn/images/BIyE6kR.png" width="450"><br/><br/>
</center>

시작하기 전에, 아래의 코드 셀을 실행하여 모든 설정을 완료하세요.

In [ ]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 49.6 MB/s eta 0:00:00


In [ ]:
import math
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim            # What you'd normally run
#from learntools.geospatial.tools import Nominatim # Just for this exercise

import folium 
from folium import Marker
from folium.plugins import MarkerCluster

이전 연습에서 사용한 `embed_map()` 함수를 사용하여 지도를 시각화합니다.

In [ ]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

# 연습

### 1) 누락된 위치에 대해 지오코딩하기

다음 코드 셀을 실행하여 캘리포니아 주에 있는 스타벅스 매장을 포함하는 DataFrame `starbucks`를 생성합니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load and preview Starbucks locations in California
starbucks = pd.read_csv("/content/drive/MyDrive/2023 데이터마이닝/dataset/starbucks_locations.csv")
starbucks.head()

,Store Number,Store Name,Address,City,Longitude,Latitude
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79


대부분의 매장은 알려진 (위도, 경도) 위치를 가지고 있습니다. 그러나 버클리 시의 모든 위치는 누락되어 있습니다.

In [ ]:
# How many rows in each column have missing values?
print(starbucks.isnull().sum())

# View rows with missing locations
rows_with_missing = starbucks[starbucks["City"]=="Berkeley"]
rows_with_missing

Store Number    0
Store Name      0
Address         0
City            0
Longitude       5
Latitude        5
dtype: int64


,Store Number,Store Name,Address,City,Longitude,Latitude
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,NaN,NaN
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,NaN,NaN
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,NaN,NaN
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,NaN,NaN
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,NaN,NaN


아래 코드 셀을 사용하여 Nominatim 지오코더로 이러한 값을 채워 넣을 수 있습니다.

참고로 튜토리얼에서는 `geopy.geocoders`의 `Nominatim()`을 사용하여 값을 지오코딩했습니다. 이것은 이 강의 외부에서도 사용할 수 있는 방법입니다.

이 연습에서는 약간 다른 함수 `learntools.geospatial.tools`의 `Nominatim()`을 사용합니다. 이 함수는 노트북 상단에서 가져온 것이며, GeoPandas의 함수와 동일하게 작동합니다.

다시 말해 다음 조건을 만족한다면:
- 노트북 상단의 import 문을 변경하지 않고,
- 아래 코드 셀에서 지오코딩 함수를 `geocode()`로 호출한다면,

원하는 대로 코드가 작동할 것입니다!

In [ ]:
# Create the geocoder
geolocator = Nominatim(user_agent="kaggle_learn")

# Your code here
def my_geocoder(row):
    point = geolocator.geocode(row).point
    return pd.Series({'Latitude': point.latitude, 'Longitude': point.longitude})

berkeley_locations = rows_with_missing.apply(lambda x: my_geocoder(x['Address']), axis=1)
starbucks.update(berkeley_locations)

### 2) 버클리 위치를 봅시다.

다음 코드 셀을 사용하여 방금 찾은 버클리의 (위도, 경도) 위치를 OpenStreetMap 스타일로 시각화해 봅시다.

In [ ]:
# Create a base map
m_2 = folium.Map(location=[37.88,-122.26], zoom_start=13)

# Your code here: Add a marker for each Berkeley location
# Add a marker for each Berkeley location
for idx, row in starbucks[starbucks["City"]=='Berkeley'].iterrows():
    Marker([row['Latitude'], row['Longitude']]).add_to(m_2)

# Uncomment to see a hint
#q_2.a.hint()

# Show the map
embed_map(m_2, 'q_2.html')

버클리의 다섯 개 위치 중 얼마나 많은 (위도, 경도) 위치가 잠재적으로 올바른 위치로 보입니다(올바른 도시에 위치해 있는지)?

In [ ]:
# Solution: All five locations appear to be correct!

### 3) 데이터를 통합합시다.

아래의 코드를 실행하여 캘리포니아 주의 각 카운티에 대한 이름, 면적(제곱 킬로미터 단위) 및 고유 ID("GEOID" 열)를 포함한 GeoDataFrame 'CA_counties'를 로드하세요. "geometry" 열에는 카운티 경계를 나타내는 폴리곤이 포함되어 있습니다.

In [ ]:
CA_counties = gpd.read_file("/content/drive/MyDrive/2023 데이터마이닝/dataset/CA_county_boundaries/CA_county_boundaries.shp")
CA_counties.crs = {'init': 'epsg:4326'}
CA_counties.head()

/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,GEOID,name,area_sqkm,geometry
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6..."
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7..."
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822..."
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3..."
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360..."


다음으로, 세 개의 DataFrame을 생성합니다:
- `CA_pop`는 각 카운티의 인구 추정치를 포함합니다.
- `CA_high_earners`는 연간 소득이 최소 $150,000인 가구 수를 포함합니다.
- `CA_median_age`는 각 카운티의 중간 연령을 포함합니다.

In [ ]:
CA_pop = pd.read_csv("/content/drive/MyDrive/2023 데이터마이닝/dataset/CA_county_population.csv", index_col="GEOID")
CA_high_earners = pd.read_csv("/content/drive/MyDrive/2023 데이터마이닝/dataset/CA_county_high_earners.csv", index_col="GEOID")
CA_median_age = pd.read_csv("/content/drive/MyDrive/2023 데이터마이닝/dataset/CA_county_median_age.csv", index_col="GEOID")

다음 코드 셀을 사용하여 `CA_counties` GeoDataFrame을 `CA_pop`, `CA_high_earners`, `CA_median_age`와 조인합니다.

결과 GeoDataFrame의 이름을 `CA_stats`로 지정하고, "GEOID", "name", "area_sqkm", "geometry", "population", "high_earners", "median_age"의 8개 열이 있는지 확인하세요.

In [ ]:
# Your code here
cols_to_add = CA_pop.join([CA_high_earners, CA_median_age]).reset_index()
CA_stats = CA_counties.merge(cols_to_add, on="GEOID")

이제 모든 데이터를 하나의 장소에 모았으므로, 여러 열을 조합하여 통계를 계산하는 것이 훨씬 쉬워졌습니다. 다음 코드 셀을 실행하여 "density" 열을 생성하여 인구 밀도를 계산하세요.

In [ ]:
CA_stats["density"] = CA_stats["population"] / CA_stats["area_sqkm"]

### 4) 어떤 지역이 유망해 보일까요?

모든 정보를 하나의 GeoDataFrame으로 병합하면 특정 기준을 충족하는 지역을 선택하는 것이 훨씬 쉬워집니다.

다음 코드 셀을 사용하여 `CA_stats` GeoDataFrame에서 일부 행(및 모든 열)을 포함하는 GeoDataFrame `sel_counties`를 생성하세요. 특히 다음 조건을 충족하는 지역을 선택해야 합니다.
- 연간 \$150,000 이상의 소득을 가진 가구가 적어도 100,000개 이상 있는 경우
- 중간 연령이 38.5보다 작은 경우
- 주민 밀도가 285 이상 (평방 킬로미터당)

또한, 선택한 지역은 다음 기준 중 하나 이상을 충족해야 합니다.
- 연간 \$150,000 이상의 소득을 가진 가구가 적어도 500,000개 이상 있는 경우
- 중간 연령이 35.5보다 작은 경우
- 주민 밀도가 1400 이상 (평방 킬로미터당)

In [ ]:
# Your code here
sel_counties = CA_stats[((CA_stats.high_earners > 100000) &
                         (CA_stats.median_age < 38.5) &
                         (CA_stats.density > 285) &
                         ((CA_stats.median_age < 35.5) |
                         (CA_stats.density > 1400) |
                         (CA_stats.high_earners > 500000)))]

### 5) 얼마나 많은 매장을 찾았나요?

다음 Starbucks Reserve Roastery 위치를 찾을 때, 선택한 지역 내의 모든 매장을 고려하고 싶습니다. 따라서 선택한 지역 내에 몇 개의 매장이 있는지 알아보려고 합니다.

이 질문에 답하기 위해, 다음 코드 셀을 실행하여 모든 스타벅스 위치를 포함하는 GeoDataFrame `starbucks_gdf`를 생성하세요.

In [ ]:
starbucks_gdf = gpd.GeoDataFrame(starbucks, geometry=gpd.points_from_xy(starbucks.Longitude, starbucks.Latitude))
starbucks_gdf.crs = {'init': 'epsg:4326'}

/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


당신이 선택한 지역에는 몇 개의 스타벅스 매장이 있나요?

In [ ]:
# Fill in your answer
locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
num_stores = len(locations_of_interest)
num_stores

1043

### 6) 매장 위치 시각화하기.

이전 질문에서 확인한 매장들의 위치를 보여주는 지도를 생성하세요.

In [ ]:
# Create a base map
m_6 = folium.Map(location=[37,-120], zoom_start=6)

# Your code here: show selected store locations
mc = MarkerCluster()

locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
for idx, row in locations_of_interest.iterrows():
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(folium.Marker([row['Latitude'], row['Longitude']]))

m_6.add_child(mc)

embed_map(m_6, 'q_6.html')